my own 


In [8]:
pip install gtts googletrans soundfile librosa numpy noisereduce

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install simpleaudio 

Note: you may need to restart the kernel to use updated packages.


In [10]:
import tkinter as tk
from tkinter import ttk
from gtts import gTTS
from googletrans import Translator
import soundfile as sf
import librosa
import numpy as np
import noisereduce
import threading

In [ ]:
#########################################################################

############################################################################

In [11]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional,Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
#import nltk
import re

In [16]:
# Specify the filename
input_file = 'holmes.txt'

# Read the contents of the file
with open(input_file, 'r', encoding='utf-8') as infile:
    data = infile.read()

In [17]:
data[:100] # view first few characters

"*Project Gutenberg's Etext of Tom Swift And His Submarine Boat*\n\n#4 in the Victor Appleton's Tom Swi"

In [4]:

# Limit data to 500000 characters
data = data[:700000]


In [5]:
# Function to remove emojis and special characters from text
def remove_emojis_and_special_characters(text):
    # Remove emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251" 
                               "]+", flags=re.UNICODE)
    # Remove Emojis from the Text
    text = emoji_pattern.sub(r'', text)

    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Remove extra spaces
    text = re.sub(' +', ' ', text)
    
    return text

In [6]:
# Preprocessing pipeline
def preprocess_pipeline(data) -> 'list':
    # Split by newline character
    sentences = data.split('\n')
    for i in range(len(sentences)):
        sentences[i] = remove_emojis_and_special_characters(sentences[i])
    # Remove leading and trailing spaces
    sentences = [s.strip() for s in sentences]
    # Drop empty sentences
    sentences = [s for s in sentences if len(s) > 0]
    # Tokenization
    tokenized = []
    for sentence in sentences:
        # Convert to lowercase
        sentence = sentence.lower()
        tokenized.append(sentence)
    return tokenized

# Tokenize sentences
tokenized_sentences = preprocess_pipeline(data)

In [7]:
"""
What is an OOV Token?
An out-of-vocabulary (OOV) token is a special token used in natural language processing (NLP) tasks to represent words that
are not present in the vocabulary of the model or tokenizer. When a word that is not in the vocabulary is encountered during
tokenization or text processing, it is replaced with the OOV token.

Why Use an OOV Token?
Using an OOV token helps handle unseen or unknown words during the training or inference phase of an NLP model. 
Instead of encountering errors or issues when encountering unknown words, the model can gracefully handle them by
representing them with the OOV token. This is particularly useful when working with real-world data where the vocabulary
of the model may not cover all possible words.
"""
# Tokenize words
tokenizer = Tokenizer(oov_token='<oov>')
tokenizer.fit_on_texts(tokenized_sentences)
total_words = len(tokenizer.word_index) + 1
# tokenizer.word_counts
# tokenizer.word_index
"""
n_gram example:
[3, 15, 8, 7, 20, 12, 6]

For the above sentece sentence, the code would generate the following n-gram sequences:

[3, 15]
[3, 15, 8]
[3, 15, 8, 7]
[3, 15, 8, 7, 20]
[3, 15, 8, 7, 20, 12]
[3, 15, 8, 7, 20, 12, 6]
"""

# Generate input sequences
input_sequences = []
for line in tokenized_sentences:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [8]:
# Creates labels with input sequences
X,labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [9]:
# Split data into training, validation, and test sets
from sklearn.model_selection import train_test_split
X_train_temp, X_val_test, y_train_temp, y_val_test = train_test_split(X, ys, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

In [2]:
# Define your model
model = Sequential()
model.add(Embedding(total_words, 100)) 
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
model.add(Dropout(0.2))
#OPTIMIZER
adam = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# Train the model
history = model.fit(X_train_temp, y_train_temp, epochs=20, validation_data=(X_val, y_val), verbose=1)

NameError: name 'Sequential' is not defined

In [ ]:
# In your training loop (after model.fit()):
print(f"Final validation accuracy: {history.history['val_accuracy'][-1]:.2f}")
# ^ This can be redirected to a log file with timestamps

In [ ]:
# Save model architecture as JSON file
from tensorflow.keras.models import model_from_json

model_json = model.to_json()
with open("lstm_model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
# Load model architecture from JSON file
from tensorflow.keras.models import model_from_json

with open("lstm_model.json", "r") as json_file:
    loaded_model_json = json_file.read()

# # Create model from loaded architecture
loaded_model = model_from_json(loaded_model_json)

print("Model architecture loaded successfully from JSON file.")


# 7. Save your trained model  <--- THIS IS THE KEY STEP
model.save('my_language_model.h5')  # Save it to a file!

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [1]:
import datetime

# Add to training script
experiment_log = {
    "date": datetime.datetime.now().isoformat(),
    "params": {
        "lstm_units": 150,
        "learning_rate": 0.01,
        "batch_size": 64
    },
    "metrics": history.history,  # From model.fit()
    "git_hash": subprocess.getoutput("git rev-parse HEAD")  # Track code version
}

# Save as JSON
import json
with open(f"experiments/exp_{datetime.datetime.now().strftime('%Y%m%d')}.json", "w") as f:
    json.dump(experiment_log, f)

NameError: name 'history' is not defined

In [ ]:
# Plot Loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot Accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
import pickle
tokenizer_file_path = r"C:\Users\nkavi\OneDrive\Desktop\play~tts\tokenizer.pickle"
try:
    
    pickle.dump(tokenizer,open(tokenizer_file_path,'wb'))
    print(f"Tokenizer successfully saved to: {tokenizer_file_path}")  # Confirm success
except Exception as e:
    print(f"Error saving tokenizer: {e}")  # Print any error message
    # Handle the error (e.g., exit, raise an exception)

code to make the tokenizer and loading the model

In [ ]:
from tensorflow.keras.models import model_from_json, load_model
import pickle

# --- Load model weights and other components if architecture loaded successfully ---
if loaded_model is not None:
    try:
        # Load the model weights from the .h5 file
        loaded_model.load_weights('my_language_model.h5')  # Load the weights into the loaded model
        # Load the tokenizer
        tokenizer_file_path = r'C:\Users\nkavi\OneDrive\Desktop\play~tts\tokenizer.pickle'
        print(f"Attempting to load tokenizer from: {tokenizer_file_path}")  # Debug
        if not os.path.exists(tokenizer_file_path):
            raise FileNotFoundError(f"Tokenizer file not found at: {tokenizer_file_path}")
        with open(tokenizer_file_path, 'rb') as handle:
            tokenizer = pickle.load(handle)
        print(f"pickle.load returned: {tokenizer}") # Debug
        if tokenizer is None:
            print("ERROR: pickle.load returned None!") #IMPORTANT
     
        max_sequence_len = 25
        print("Model architecture, weights, and tokenizer loaded successfully.")
    except Exception as e:
        print(f"Error loading model weights or tokenizer: {e}")
        # Optionally, you could set loaded_model = None here if loading weights fails
        #  depending on how you want your program to handle this scenario.
        tokenizer = None
        max_sequence_len = None
else:
  tokenizer = None
  max_sequence_len = None
print(f"Tokenizer: {tokenizer}")
print(f"Max Sequence Length: {max_sequence_len}")

In [ ]:
def predict_top_five_words(model, tokenizer, seed_text):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    top_five_indexes = np.argsort(predicted[0])[::-1][:5]
    top_five_words = []
    for index in top_five_indexes:
        for word, idx in tokenizer.word_index.items():
            if idx == index:
                top_five_words.append(word)
                break
    return top_five_words

using with enhanced output feature

In [ ]:
#################################################################
GOOD JOB @

In [ ]:
pip install gtts pydub

In [ ]:
pip install langdetect


In [ ]:
pip install simpleaudio

In [ ]:
pip install ffmpeg

In [ ]:
pip install ffprobe

In [ ]:
from pydub import AudioSegment
AudioSegment.converter =r"c:\\Users\\nkavi\\anaconda3\\Lib\\site-packages\\ffmpeg"  # Update with correct path

In [ ]:
import ffmpeg
print(ffmpeg)


this ffmpeg didnt denied access to my laptop

In [ ]:
import pygame
import tkinter as tk
import threading

In [ ]:
pip install Pillow

In [ ]:
pip install scikit-image

In [ ]:
pip install pyimage

In [ ]:
#main code after final aboves are the background and speak button checking

perfefct perfect perfect @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

In [ ]:
pip install mutagen

In [ ]:
pip install pydub pygame mutagen gtts

trying to load the saved file

In [ ]:
from playsound import playsound  # Add this import with your other imports

In [ ]:
pip install gTTS pydub simpleaudio


modern gui 

In [ ]:
pip install mutagen

In [ ]:
import os
import subprocess

In [1]:
def open_player():
    """Open the last generated audio file in system default player"""
    global current_audio_file
    
    if not current_audio_file or not os.path.exists(current_audio_file):
        messagebox.showwarning("No Audio File", "No audio file has been generated yet. Please generate speech first.")
        return
    
    try:
        # Windows
        if os.name == 'nt':
            os.startfile(current_audio_file)
        # macOS
        elif sys.platform == 'darwin':
            subprocess.run(['open', current_audio_file])
        # Linux/Unix
        else:
            subprocess.run(['xdg-open', current_audio_file])
    except Exception as e:
        messagebox.showerror("Error", f"Could not open audio player: {str(e)}")

In [4]:
def open_player():
    """Open the last generated audio file in system default player"""
    global current_audio_file
    
    if not current_audio_file or not os.path.exists(current_audio_file):
        messagebox.showwarning("No Audio File", "No audio file has been generated yet. Please generate speech first.")
        return
    
    try:
        # Windows
        if os.name == 'nt':
            os.startfile(current_audio_file)
        # macOS
        elif sys.platform == 'darwin':
            subprocess.run(['open', current_audio_file])
        # Linux/Unix
        else:
            subprocess.run(['xdg-open', current_audio_file])
    except Exception as e:
        messagebox.showerror("Error", f"Could not open audio player: {str(e)}")

In [1]:
pip install socket

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement socket (from versions: none)
ERROR: No matching distribution found for socket


checkinh

In [5]:
import tkinter as tk
from tkinter import ttk, messagebox, Button, scrolledtext, Toplevel, filedialog
from tkinter.font import Font
import googletrans
import threading
import time
import random
import numpy as np
import pickle
import os
import subprocess
import sys
import socket
import requests
from gtts import gTTS
import pygame
import tempfile
from pydub import AudioSegment
import wave
import simpleaudio as sa
from langdetect import detect, LangDetectException
import traceback
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ====== STYLING CONSTANTS ======
BG_COLOR = "#1a1a1a"  # Dark background
FG_COLOR = "#ffffff"  # White text
ACCENT_COLOR = "#00ff99"  # Bright green accent
SECONDARY_COLOR = "#333333"  # Dark gray for secondary elements
TEXT_BG = "#2d2d2d"  # Slightly lighter than BG for text areas
BUTTON_ACTIVE = "#007755"  # Darker green for active buttons
FONT_FAMILY = "Segoe UI"  # Modern, clean font
BUTTON_RED = "#ff4444"  # For stop button
SIDEBAR_COLOR = "#222222"  # Darker sidebar color

# ====== STYLE CONFIGURATION ======
def configure_styles():
    style = ttk.Style()
    style.theme_use('clam')
    
    # Main window style
    style.configure('.', 
                   background=BG_COLOR,
                   foreground=FG_COLOR,
                   font=(FONT_FAMILY, 10))
    
    # Button styles
    style.configure('TButton', 
                   background=SECONDARY_COLOR,
                   foreground=FG_COLOR,
                   borderwidth=1,
                   relief='raised',
                   padding=5)
    style.map('TButton',
              background=[('active', BUTTON_ACTIVE), ('pressed', BUTTON_ACTIVE)],
              relief=[('pressed', 'sunken'), ('!pressed', 'raised')])
    
    # Combobox style
    style.configure('TCombobox',
                   fieldbackground=TEXT_BG,
                   background=SECONDARY_COLOR,
                   foreground=FG_COLOR,
                   selectbackground=ACCENT_COLOR,
                   selectforeground=FG_COLOR)
    style.map('TCombobox',
              fieldbackground=[('readonly', TEXT_BG)],
              selectbackground=[('readonly', ACCENT_COLOR)])
    
    # Scale style
    style.configure('Horizontal.TScale',
                   background=BG_COLOR,
                   troughcolor=SECONDARY_COLOR,
                   sliderrelief='raised',
                   sliderlength=15)
    
    # Scrollbar style
    style.configure('Vertical.TScrollbar',
                   background=SECONDARY_COLOR,
                   troughcolor=BG_COLOR,
                   arrowcolor=FG_COLOR,
                   bordercolor=BG_COLOR)
    
    # Label styles
    style.configure('Header.TLabel',
                   font=(FONT_FAMILY, 18, 'bold'),
                   foreground=ACCENT_COLOR)
    style.configure('Section.TLabel',
                   font=(FONT_FAMILY, 12, 'bold'),
                   foreground=FG_COLOR)
    style.configure('Normal.TLabel',
                   font=(FONT_FAMILY, 10),
                   foreground=FG_COLOR)
    style.configure('Sidebar.TButton',
                   background=SIDEBAR_COLOR,
                   foreground=FG_COLOR,
                   borderwidth=0,
                   relief='flat',
                   padding=10,
                   anchor='w')
    style.map('Sidebar.TButton',
              background=[('active', BUTTON_ACTIVE), ('pressed', BUTTON_ACTIVE)])

# Global variables
floating_predictions_window = None
model = None
tokenizer = None
max_sequence_len = 0
current_audio_file = None
volume_level = 80
speaking_speed_multiplier = 1.0
STOP_REQUESTED = False
AUDIO_FILE_DIR = r"C:\Users\nkavi\OneDrive\Desktop\play~tts"

if not os.path.exists(AUDIO_FILE_DIR):
    os.makedirs(AUDIO_FILE_DIR)

try:
    if os.path.exists('my_language_model.h5') and os.path.exists('tokenizer.pickle'):
        model = load_model('my_language_model.h5')
        with open('tokenizer.pickle', 'rb') as handle:
            tokenizer = pickle.load(handle)
        max_sequence_len = 25
    else:
        print("Warning: Model files not found. Prediction features will be disabled.")
except Exception as e:
    print(f"Error loading model: {str(e)}")
    model = None
    tokenizer = None
    max_sequence_len = 0

# Initialize translator
translator = googletrans.Translator()

# List of supported languages and their codes
language_options = {
    "Afrikaans": "af", "Arabic": "ar", "Bulgarian": "bg", "Bengali": "bn",
    "Bosnian": "bs", "Catalan": "ca", "Czech": "cs", "Welsh": "cy",
    "Danish": "da", "German": "de", "Greek": "el", "English": "en",
    "Spanish": "es", "Estonian": "et", "Finnish": "fi", "French": "fr",
    "Gujarati": "gu", "Hindi": "hi", "Croatian": "hr", "Hungarian": "hu",
    "Indonesian": "id", "Icelandic": "is", "Italian": "it", "Hebrew": "iw",
    "Japanese": "ja", "Javanese": "jw", "Khmer": "km", "Kannada": "kn",
    "Korean": "ko", "Latin": "la", "Latvian": "lv", "Malayalam": "ml",
    "Marathi": "mr", "Malay": "ms", "Burmese": "my", "Nepali": "ne",
    "Dutch": "nl", "Norwegian": "no", "Polish": "pl", "Portuguese": "pt",
    "Romanian": "ro", "Russian": "ru", "Sinhala": "si", "Slovak": "sk",
    "Albanian": "sq", "Serbian": "sr", "Sundanese": "su", "Swedish": "sv",
    "Swahili": "sw", "Tamil": "ta", "Telugu": "te", "Thai": "th",
    "Filipino": "tl", "Turkish": "tr", "Ukrainian": "uk", "Urdu": "ur",
    "Vietnamese": "vi", "Chinese (Simplified)": "zh-CN", "Chinese (Mandarin/Taiwan)": "zh-TW"
}

# Create the main window
window = tk.Tk()
window.title("PLAY~TTS")
window.iconbitmap("tts_icon.ico")
window.geometry("900x700")
window.configure(bg=BG_COLOR)

# Configure styles first
configure_styles()

# ====== SIDEBAR ======
sidebar = tk.Frame(window, bg=SIDEBAR_COLOR, width=150)
sidebar.pack(side="left", fill="y", expand=False)

# Sidebar buttons
def create_sidebar_button(parent, text, command):
    btn = ttk.Button(parent, text=text, style='Sidebar.TButton', command=command)
    btn.pack(fill="x", pady=(5, 0), padx=5)
    return btn

# Will be populated after tab frames are created
sidebar_buttons = []
 
# ====== NOTEBOOK (TABS) ======
notebook = ttk.Notebook(window)
notebook.pack(side="right", fill="both", expand=True)

# Tab frames
welcome_tab = tk.Frame(notebook, bg=BG_COLOR)
home_tab = tk.Frame(notebook, bg=BG_COLOR)
about_tab = tk.Frame(notebook, bg=BG_COLOR)

notebook.add(welcome_tab, text="Welcome")
notebook.add(home_tab, text="Home")
notebook.add(about_tab, text="About")

# Function to switch tabs
def switch_tab(tab_index):
    notebook.select(tab_index)
    # Update button states
    for i, btn in enumerate(sidebar_buttons):
        if i == tab_index:
            btn.config(style='Sidebar.TButton.Active')
        else:
            btn.config(style='Sidebar.TButton')

# Create sidebar buttons after tabs are created
sidebar_buttons.append(create_sidebar_button(sidebar, "Welcome", lambda: switch_tab(0)))
sidebar_buttons.append(create_sidebar_button(sidebar, "Home TTS", lambda: switch_tab(1)))
sidebar_buttons.append(create_sidebar_button(sidebar, "About", lambda: switch_tab(2)))
sidebar_buttons.append(create_sidebar_button(sidebar, "🎵 Open Audio", open_player)) # 👈 Your new button here


# Custom style for active sidebar button
style = ttk.Style()
style.configure('Sidebar.TButton.Active', 
               background=ACCENT_COLOR,
               foreground="#000000",
               borderwidth=0,
               relief='flat',
               padding=10,
               anchor='w')

# ====== WELCOME TAB CONTENT ======
welcome_frame = tk.Frame(welcome_tab, bg=BG_COLOR)
welcome_frame.pack(fill="both", expand=True, padx=20, pady=20)

welcome_label = tk.Label(welcome_frame, 
                        text="Welcome to PLAY~TTS", 
                        font=(FONT_FAMILY, 24, 'bold'),
                        bg=BG_COLOR,
                        fg=ACCENT_COLOR)
welcome_label.pack(pady=(50, 20))

welcome_text = tk.Label(welcome_frame,
                       text="Your personal text-to-speech companion\n\n"
                            "Translate and listen to text in multiple languages with ease.\n"
                            "Use the Home tab for TTS functionality or explore the About tab\n"
                            "to learn more about this application.",
                       font=(FONT_FAMILY, 12),
                       bg=BG_COLOR,
                       fg=FG_COLOR,
                       justify="center")
welcome_text.pack(pady=10)

# ====== HOME TAB CONTENT (TTS FUNCTIONALITY) ======
# Header (styled to match new theme)
header_font = Font(family=FONT_FAMILY, size=18, weight='bold')
header = tk.Label(home_tab, 
                 text="PLAY~TTS", 
                 font=header_font,
                 bg=BG_COLOR,
                 fg=ACCENT_COLOR)
header.pack(pady=(10, 20))

# Settings Section (styled to match new theme)
settings_frame = tk.Frame(home_tab, bg=BG_COLOR)
settings_frame.pack(pady=(0, 15))

settings_label = tk.Label(settings_frame, 
                        text="Settings", 
                        font=(FONT_FAMILY, 12, 'bold'),
                        bg=BG_COLOR,
                        fg=FG_COLOR)
settings_label.pack(anchor='w', pady=(0, 10))

grid_frame = tk.Frame(settings_frame, bg=BG_COLOR)
grid_frame.pack()

# Language Dropdown
language_label = tk.Label(grid_frame, 
                        text="Target Language:", 
                        bg=BG_COLOR,
                        fg=FG_COLOR)
language_label.grid(row=0, column=0, sticky='w', pady=2)

language_var = tk.StringVar()
language_dropdown = ttk.Combobox(grid_frame, 
                               textvariable=language_var, 
                               values=sorted(language_options.keys()),
                               style='TCombobox')
language_dropdown.set("English")
language_dropdown.grid(row=0, column=1, sticky='ew', pady=2, padx=(5, 0))

# Voice Dropdown
voice_label = tk.Label(grid_frame, 
                     text="Voice Gender:", 
                     bg=BG_COLOR,
                     fg=FG_COLOR)
voice_label.grid(row=1, column=0, sticky='w', pady=2)

voice_var = tk.StringVar()
voice_dropdown = ttk.Combobox(grid_frame, 
                            textvariable=voice_var, 
                            values=["Female"],
                            style='TCombobox')
voice_dropdown.set("Female")
voice_dropdown.grid(row=1, column=1, sticky='ew', pady=2, padx=(5, 0))

# Volume Scale
volume_label = tk.Label(grid_frame, 
                      text="Volume:", 
                      bg=BG_COLOR,
                      fg=FG_COLOR)
volume_label.grid(row=3, column=0, sticky='w', pady=2)

volume_scale = ttk.Scale(grid_frame, 
                       from_=0, 
                       to=100, 
                       orient=tk.HORIZONTAL,
                       style='Horizontal.TScale')
volume_scale.set(volume_level)
volume_scale.grid(row=3, column=1, sticky='ew', pady=2, padx=(5, 0))



def change_volume(value):
    try:
       global volume_level
       volume_level = int(float(value))
       print(f"Volume set to {volume_level}")
    except ValueError as e:
       print(f"Volume conversion error: {e}")
volume_scale.config(command=change_volume)

# Text Areas
text_frame = tk.Frame(home_tab, bg=BG_COLOR)
text_frame.pack(fill='both', expand=True, padx=10, pady=(0, 15))

# Input Text
input_label = tk.Label(text_frame, 
                     text="Input Text", 
                     bg=BG_COLOR,
                     fg=FG_COLOR)
input_label.pack(anchor='w', pady=(0, 5))

text_area = scrolledtext.ScrolledText(text_frame, 
                                    height=8, 
                                    width=50, 
                                    wrap=tk.WORD,
                                    bg=TEXT_BG,
                                    fg=FG_COLOR,
                                    insertbackground=FG_COLOR,
                                    selectbackground=ACCENT_COLOR,
                                    selectforeground=FG_COLOR,
                                    relief='flat',
                                    padx=10,
                                    pady=10)
text_area.pack(fill='both', expand=True)
text_area.insert(tk.END, "Type or paste text here.")

def clear_input_text(event):
    if text_area.get("1.0", "end-1c").strip() == "Type or paste text here.":
        text_area.delete("1.0", tk.END)
        text_area.unbind('<FocusIn>', clear_input_text_id)

clear_input_text_id = text_area.bind('<FocusIn>', clear_input_text)

# Output Text
output_label = tk.Label(text_frame, 
                      text="Translated Text", 
                      bg=BG_COLOR,
                      fg=FG_COLOR)
output_label.pack(anchor='w', pady=(10, 5))

output_text = tk.Text(text_frame, 
                    height=8, 
                    width=50, 
                    state=tk.NORMAL, 
                    wrap=tk.WORD,
                    bg=TEXT_BG,
                    fg=FG_COLOR,
                    insertbackground=FG_COLOR,
                    selectbackground=ACCENT_COLOR,
                    selectforeground=FG_COLOR,
                    relief='flat',
                    padx=10,
                    pady=10)
output_text.pack(fill='both', expand=True)

# Add copy button below the output text
def add_copy_button(parent, text_widget):
    button_frame = tk.Frame(parent, bg=BG_COLOR)
    button_frame.pack(pady=(5, 0), anchor='w')
    
    copy_btn = tk.Button(button_frame, 
                        text="Copy", 
                        font=(FONT_FAMILY, 9),
                        borderwidth=0,
                        relief='flat',
                        bg=SECONDARY_COLOR,
                        fg=FG_COLOR,
                        activebackground=BUTTON_ACTIVE,
                        activeforeground=FG_COLOR,
                        command=lambda: copy_text(text_widget, copy_btn))
    copy_btn.pack(side='left', padx=(0, 5))
    
    return copy_btn

def copy_text(text_widget, button):
    """Copy text to clipboard with visual feedback"""
    try:
        text = text_widget.get("1.0", "end-1c")
        if text.strip():
            text_widget.clipboard_clear()
            text_widget.clipboard_append(text)
            
            # Visual feedback
            original_text = button.cget("text")
            button.config(text="✓ Copied!", fg=ACCENT_COLOR)
            
            # Revert after 1.5 seconds
            text_widget.after(1500, lambda: button.config(
                text=original_text, 
                fg=FG_COLOR
            ))
    except Exception as e:
        print(f"Copy error: {e}")

add_copy_button(text_frame, output_text)

# Configure speaking highlight tag
output_text.tag_configure("speaking", background=ACCENT_COLOR, foreground="#000000")

# Action Buttons
button_frame = tk.Frame(home_tab, bg=BG_COLOR)
button_frame.pack(pady=(10, 20))

translate_btn = tk.Button(button_frame, 
                     text="TRANSLATE", 
                     bg=ACCENT_COLOR,
                     fg="#000000",
                     activebackground=BUTTON_ACTIVE,
                     activeforeground=FG_COLOR,
                     relief='flat',
                     borderwidth=0,
                     padx=20,
                     pady=8,
                     font=(FONT_FAMILY, 10, 'bold'),
                     command=lambda: translate_text_output())
translate_btn.pack(side=tk.LEFT, padx=5, fill='x', expand=True)

speak_btn = tk.Button(button_frame, 
                     text="SPEAK", 
                     bg=ACCENT_COLOR,
                     fg="#000000",
                     activebackground=BUTTON_ACTIVE,
                     activeforeground=FG_COLOR,
                     relief='flat',
                     borderwidth=0,
                     padx=20,
                     pady=8,
                     font=(FONT_FAMILY, 10, 'bold'),
                     command=lambda: speaknow())
speak_btn.pack(side=tk.LEFT, padx=5, fill='x', expand=True)

stop_btn = tk.Button(button_frame, 
                    text="STOP", 
                    bg=BUTTON_RED,
                    fg=FG_COLOR,
                    activebackground="#cc0000",
                    activeforeground=FG_COLOR,
                    relief='flat',
                    borderwidth=0,
                    padx=20,
                    pady=8,
                    font=(FONT_FAMILY, 10, 'bold'),
                    command=lambda: stop_playback())
stop_btn.pack(side=tk.LEFT, padx=5, fill='x', expand=True)

# Status Bar
status_frame = tk.Frame(home_tab, bg=SECONDARY_COLOR, height=25)
status_frame.pack(fill='x', side='bottom')

quota_label = tk.Label(status_frame, 
                     text="0 / 500 \u00A0\u00A0\u00A0\u00A0Total quota remaining: 50", 
                     bg=SECONDARY_COLOR,
                     fg=FG_COLOR)
quota_label.pack(side='left', padx=10)

# ====== ABOUT TAB CONTENT ======
about_frame = tk.Frame(about_tab, bg=BG_COLOR)
about_frame.pack(fill="both", expand=True, padx=20, pady=20)

about_label = tk.Label(about_frame, 
                     text="About PLAY~TTS", 
                     font=(FONT_FAMILY, 24, 'bold'),
                     bg=BG_COLOR,
                     fg=ACCENT_COLOR)
about_label.pack(pady=(20, 10))

about_text = tk.Text(about_frame,
                   wrap=tk.WORD,
                   bg=BG_COLOR,
                   fg=FG_COLOR,
                   font=(FONT_FAMILY, 11),
                   padx=10,
                   pady=10,
                   relief='flat',
                   height=15)
about_text.pack(fill="both", expand=True)

about_content = """
PLAY~TTS is a powerful text-to-speech application that allows you to:

• Translate text between multiple languages
• Listen to the translated text with natural-sounding voices
• Adjust voice speed and volume for optimal listening
• Predict words as you type for faster input

This application was developed to provide an easy-to-use interface for 
text-to-speech conversion with translation capabilities. 

Version: 1.0
Release Date: 10-09-2025

Features:
- Support for 50+ languages
- Voice gender selection
- Volume control
- Word prediction
- Clean, modern interface

For support or feedback, please contact the developer.
"""
about_text.insert(tk.END, about_content)
about_text.config(state=tk.DISABLED)

# Add a frame for additional info
info_frame = tk.Frame(about_frame, bg=BG_COLOR)
info_frame.pack(fill="x", pady=(10, 0))

# Developer info
dev_label = tk.Label(info_frame,
                   text="Developer: KAVIYA N\nContact: nkaviya419@gmail.com",
                   bg=BG_COLOR,
                   fg=FG_COLOR,
                   font=(FONT_FAMILY, 10),
                   justify="left")
dev_label.pack(side="left", padx=10)

# Version info
version_label = tk.Label(info_frame,
                       text=f"Version: 1.0\nLast Updated: 10-09-2025",
                       bg=BG_COLOR,
                       fg=FG_COLOR,
                       font=(FONT_FAMILY, 10),
                       justify="right")
version_label.pack(side="right", padx=10)


# ====== STYLE CONFIGURATION ======
def configure_styles():
    style = ttk.Style()
    style.theme_use('clam')
    
    # Configure base style first
    style.configure('Sidebar.TButton', 
                   background=SIDEBAR_COLOR,
                   foreground=FG_COLOR,
                   borderwidth=0,
                   relief='flat',
                   padding=10,
                   anchor='w',
                   font=(FONT_FAMILY, 10))
    
    # Then configure active variant
    style.configure('Sidebar.TButton.Active', 
                   background=ACCENT_COLOR,
                   foreground="#000000",
                   borderwidth=0,
                   relief='flat',
                   padding=10,
                   anchor='w',
                   font=(FONT_FAMILY, 10))
    
    # Add this mapping to handle interactions
    style.map('Sidebar.TButton',
             background=[('active', BUTTON_ACTIVE),
                         ('selected', ACCENT_COLOR)])

# Modify the switch_tab function
def switch_tab(tab_index):
    notebook.select(tab_index)
    # Update button states using ttk state management
    for i, btn in enumerate(sidebar_buttons):
        if i == tab_index:
            btn.state(['selected'])
        else:
            btn.state(['!selected'])


def open_player():
    """Open the last generated audio file in system default player"""
    global current_audio_file
    
    if not current_audio_file or not os.path.exists(current_audio_file):
        messagebox.showwarning("No Audio File", "No audio file has been generated yet. Please generate speech first.")
        return
    
    try:
        # Windows
        if os.name == 'nt':
            os.startfile(current_audio_file)
        # macOS
        elif sys.platform == 'darwin':
            subprocess.run(['open', current_audio_file])
        # Linux/Unix
        else:
            subprocess.run(['xdg-open', current_audio_file])
    except Exception as e:
        messagebox.showerror("Error", f"Could not open audio player: {str(e)}")


# ====== ORIGINAL FUNCTIONALITY ======
def check_internet_connection():
    """Check if there's an active internet connection."""
    try:
        socket.create_connection(("8.8.8.8", 53), timeout=3)
        return True
    except OSError:
        pass
    try:
        requests.get("https://www.google.com", timeout=3)
        return True
    except requests.exceptions.RequestException:
        return False

def predict_top_five_words(model, tokenizer, seed_text, max_sequence_len):
    try:
        if model is None or tokenizer is None:
            return []

        if not seed_text.strip():
            return []

        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        top_five_indexes = np.argsort(predicted[0])[::-1][:5]

        top_five_words = []
        for index in top_five_indexes:
            for word, idx in tokenizer.word_index.items():
                if idx == index:
                    top_five_words.append(word)
                    break
        return top_five_words[:5]
    except Exception as e:
        print(f"Error in prediction: {str(e)}")
        return []

def show_predictions(event=None):
    if model is None or tokenizer is None:
        return

    try:
        current_text = text_area.get("1.0", tk.END).strip()
        if not current_text:
            hide_predictions()
            return

        cursor_index = text_area.index(tk.INSERT)
        line, char = map(int, cursor_index.split('.'))
        prefix_text = text_area.get(f"{line}.0", cursor_index).strip()

        if prefix_text:
            top_words = predict_top_five_words(model, tokenizer, prefix_text, max_sequence_len)
            if top_words:
                display_floating_predictions(top_words, cursor_index)
            else:
                hide_predictions()
        else:
            hide_predictions()
    except Exception as e:
        print(f"Error showing predictions: {str(e)}")
        hide_predictions()

def display_floating_predictions(words, cursor_index):
    global floating_predictions_window

    hide_predictions()

    if words:
        try:
            floating_predictions_window = Toplevel(window)
            floating_predictions_window.transient(window)
            floating_predictions_window.overrideredirect(True)

            x, y, _, _ = text_area.bbox(cursor_index)
            if x is None or y is None:
                x, y = 0, 20

            text_x, text_y = text_area.winfo_rootx(), text_area.winfo_rooty()
            prediction_x = text_x + x
            prediction_y = text_y + y + 20

            floating_predictions_window.geometry(f"+{prediction_x}+{prediction_y}")

            for i, word in enumerate(words):
                btn = Button(floating_predictions_window, text=word,
                             command=lambda w=word: add_word(w),
                             relief=tk.FLAT, bg="#f0f0f0", padx=2, pady=1)
                btn.pack(side="left", padx=1, pady=1)

            floating_predictions_window.deiconify()
            floating_predictions_window.lift()

            floating_predictions_window.bind("<Escape>", lambda e: hide_predictions())
        except Exception as e:
            print(f"Error displaying predictions: {str(e)}")
            if floating_predictions_window:
                floating_predictions_window.destroy()
            floating_predictions_window = None

def hide_predictions(event=None):
    global floating_predictions_window
    if floating_predictions_window:
        try:
            floating_predictions_window.destroy()
        except:
            pass
        floating_predictions_window = None

def add_word(word):
    try:
        cursor_index_before_insert = text_area.index(tk.INSERT)
        text_area.insert(cursor_index_before_insert, f" {word}")

        line, char = map(int, cursor_index_before_insert.split('.'))
        new_char = char + len(word) + 1
        text_area.mark_set(tk.INSERT, f"{line}.{new_char}")
        text_area.see(tk.INSERT)

        window.after(100, show_predictions)
        hide_predictions()
    except Exception as e:
        print(f"Error adding word: {str(e)}")

def translate_text_output():
    """Translates the input text and displays it in the output text box."""
    if not check_internet_connection():
        messagebox.showerror("Network Error", "No internet connection. Please check your network and try again.")
        return

    text = text_area.get("1.0", tk.END).strip()
    if not text:
        messagebox.showwarning("Input Error", "Please enter some text to translate.")
        return

    language = language_var.get()
    if language not in language_options:
        messagebox.showwarning("Language Error", "Selected language is not supported.")
        return

    try:
        translated_text = translator.translate(text, dest=language_options[language]).text
        output_text.config(state=tk.NORMAL)
        output_text.delete("1.0", tk.END)
        output_text.insert(tk.END, translated_text)
        output_text.config(state=tk.NORMAL)
    except Exception as e:
        error_message = f"Translation failed: {str(e)}"
        if "timed out" in str(e).lower():
            error_message = "Translation timed out. Please check your internet connection."
        messagebox.showerror("Translation Error", error_message)
        output_text.config(state=tk.NORMAL)
        output_text.delete("1.0", tk.END)
        output_text.insert(tk.END, "Translation failed - check network")
        output_text.config(state=tk.NORMAL)

def speak_humanized(text, target_textbox=None):
    """Speaks text with gTTS and highlights words as they're spoken."""
    global current_audio_file, STOP_REQUESTED, AUDIO_FILE_DIR, volume_level, speaking_speed_multiplier

    if not text.strip():
        return

    STOP_REQUESTED = False  # Reset the stop request flag

    try:
        print(f"Current speed multiplier: {speaking_speed_multiplier}")
        target_lang = language_options.get(language_var.get(), "en")
        speech_speed = False  # gTTS handles the slow parameter, the multiplier is for playsound

        # Create a unique filename in the specified directory
        filename = f"translation_{time.time()}.mp3"
        filepath = os.path.join(AUDIO_FILE_DIR, filename)
        current_audio_file = filepath
        
        # Generate speech with gTTS (as MP3)
        tts = gTTS(text=text, lang=target_lang, slow=speech_speed)
        tts.save(filepath)
        
        # Initialize pygame mixer if not already done
        if not pygame.mixer.get_init():
            pygame.mixer.init()
        
        # Load and adjust speed using pydub (if speed is not normal)
        if speaking_speed_multiplier != 1.0:
            try:
                sound = AudioSegment.from_mp3(filepath)
                sound = sound.speedup(playback_speed=speaking_speed_multiplier)
                adjusted_filepath = os.path.join(AUDIO_FILE_DIR, f"adjusted_{filename}")
                sound.export(adjusted_filepath, format="mp3")
                pygame.mixer.music.load(adjusted_filepath)
                current_audio_file = adjusted_filepath
            except Exception as e:
                print(f"Speed adjustment failed: {e}")
                pygame.mixer.music.load(filepath)
        else:
            pygame.mixer.music.load(filepath)
        
        # Set volume (0.0 to 1.0)
        pygame.mixer.music.set_volume(volume_level / 100)
        pygame.mixer.music.play()

        # Split text for highlighting
        words = text.split()
        word_positions = []
        current_pos = 0
        # Find positions of each word
        for word in words:
            start = text.find(word, current_pos)
            if start == -1:
                continue
            end = start + len(word)
            word_positions.append((start, end))
            current_pos = end
        
        def play_and_highlight():
            try:
                # Calculate base duration per word (adjusted by speed multiplier)
                base_duration_per_word = 0.3  # Default base duration per word (seconds)
                duration_per_word = base_duration_per_word / speaking_speed_multiplier
            
                try:
                    # Try to get duration using mutagen as fallback
                    from mutagen.mp3 import MP3
                    audio = MP3(filepath)
                    audio_length = audio.info.length
                    duration_per_word = audio_length / len(words) if words else 0.3
                except:
                    # Fallback calculation
                    avg_word_length = sum(len(word) for word in words) / len(words) if words else 5
                    duration_per_word = (avg_word_length * 0.05) / speaking_speed_multiplier
        
                # Highlight words if we have a textbox
                if target_textbox:
                    for (start, end), word in zip(word_positions, words):
                        if STOP_REQUESTED:
                            break
                        target_textbox.tag_add("speaking", f"1.{start}", f"1.{end}")
                        target_textbox.see(f"1.{start}")
                        time.sleep(duration_per_word)
                        target_textbox.tag_remove("speaking", f"1.{start}", f"1.{end}")
                # Wait for playback to finish if not stopped
                while pygame.mixer.music.get_busy() and not STOP_REQUESTED:
                    time.sleep(0.1)

            except Exception as e:
                messagebox.showerror("Playback Error", f"Error during playback: {str(e)}")
                print(traceback.format_exc())
            finally:
                # Clean up audio file
                if not STOP_REQUESTED and current_audio_file and os.path.exists(current_audio_file):
                    for f in [filepath, current_audio_file]:
                        if f and os.path.exists(f) and f != filepath:
                            try:
                                os.remove(f)
                            except:
                                pass
                
                pygame.mixer.music.stop()
        threading.Thread(target=play_and_highlight, daemon=True).start()

    except Exception as e:
        messagebox.showerror("TTS Error", f"Error generating speech: {str(e)}")
        print(traceback.format_exc())
        # Clean up if file was created but error occurred
        if current_audio_file and os.path.exists(current_audio_file):
            try:
                os.remove(current_audio_file)
            except:
                pass

def speaknow():
    """Function to speak the translated text with selected options."""
    translated_text = output_text.get("1.0", "end-1c").strip()
    if not translated_text:
        messagebox.showwarning("No Text", "No translated text available to speak.")
        return

    # Use a thread to prevent GUI freezing
    threading.Thread(
        target=speak_humanized,
        args=(translated_text, output_text),  # Pass the output_text
        daemon=True
    ).start()

def stop_playback():
    """Stop the current audio playback."""
    global current_audio_file, STOP_REQUESTED
    STOP_REQUESTED = True
    try:
        if current_audio_file and os.path.exists(current_audio_file):
            os.remove(current_audio_file)
            current_audio_file = None
    except Exception as e:
        print(f"Error stopping playback: {str(e)}")

# Bind text input events
text_area.bind("<KeyRelease>", show_predictions)
text_area.bind("<Button-1>", show_predictions)
text_area.bind("<FocusOut>", hide_predictions)

def on_closing():
    """Handles the window close event."""
    if messagebox.askokcancel("Quit", "Do you want to quit?"):
        stop_playback()
        window.destroy()

window.protocol("WM_DELETE_WINDOW", on_closing)

# Start the main loop
if __name__ == "__main__":
    try:
        # Select the Home tab by default
        switch_tab(1)
        window.mainloop()
    except Exception as e:
        messagebox.showerror("Fatal Error", f"The application encountered an error and will close: {str(e)}")
        print(traceback.format_exc())

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\nkavi\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\nkavi\AppData\Local\Temp\ipykernel_19956\128773362.py", line 914, in on_closing
    window.destroy()
  File "c:\Users\nkavi\anaconda3\Lib\tkinter\__init__.py", line 2390, in destroy
    self.tk.call('destroy', self._w)
_tkinter.TclError: can't invoke "destroy" command: application has been destroyed
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\nkavi\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\nkavi\AppData\Local\Temp\ipykernel_19956\128773362.py", line 914, in on_closing
    window.destroy()
  File "c:\Users\nkavi\anaconda3\Lib\tkinter\__init__.py", line 2390, in destroy
    self.tk.call('destroy', self._w)
_tkinter.TclError: can't invoke "destroy